<a href="https://colab.research.google.com/github/tuonglab/BIOL3003_workshop/blob/master/notebook/BIOL3003_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UQ BIOL3003 Workshop: Introduction to Single-Cell RNA Sequencing Analysis with Python

## OVERALL OBJECTIVE

Students will work in groups of 2. Each group will be provided with a single-cell data of mouse spleens from n=8 12-week old mice that were sent to the International Space Station by NASA (space; n=4) stayed at ground control (earth; n=4). The data has undergone initial pre-processing to add sample metadata and down sampling to ~3000 cells per sample. The main exercise is to **annotate the major immune cell types found in spleen**. Time permitting, **we will also explore if space travel led to differential abundance of the immune cell types**.


## STEP 1: Installing pre-requisite software and downloading data

This workshop will get you started on single-cell analysis in Python using [`Scanpy`](https://scanpy.readthedocs.io/en/stable/), the toolkit for analysing single-cell gene expression data.

<a href="https://scanpy.readthedocs.io/en/stable/"><img src="https://scanpy.readthedocs.io/en/stable/_static/Scanpy_Logo_BrightFG.svg" alt="anndata_schema" width="100">


In [ ]:
# install scanpy
!pip install scanpy[leiden]
# clone the repository so that we have all the data and notebooks ready to go
!git clone https://github.com/tuonglab/BIOL3003_workshop.git


## STEP 2: Loading up software

In [ ]:
import os
import pandas as pd
import scanpy as sc

# change to working directory
os.chdir("BIOL3003_workshop")
# Print package versions
sc.logging.print_header()

## STEP 3: Reading the data and inspect the object


In [ ]:
adata = sc.read_h5ad("data/mouse_spleen.h5ad")
# Print the summary of the object
adata

In [ ]:
# View the cell-level metadata
adata.obs

In [ ]:
# View the gene-level metadata
adata.var

## STEP 4: Perform standard quality control

### <u>4A: Tabulate %mitochondrial content</u>
High mitochondrial content is often associated with poor quality cells. We can calculate the percentage of mitochondrial genes in each cell and plot it.


In [ ]:
# mitochondrial genes starts with "mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True, log1p=True)
sc.pl.violin(
    adata,
    [
        "n_genes_by_counts",  # the number of genes per cell
        "total_counts",  # the total UMI counts per cell
        "pct_counts_mt",  # the percentage of counts in mitochondrial genes
    ],
    jitter=0.4,
    multi_panel=True,
)

### <u>4B: Filter low quality cells</u>
Before performing analysis, we ensure the dataset meets minimal quality standards.


In [ ]:
# filter cells if they do not express at least 200 genes
sc.pp.filter_cells(adata, min_genes=200)
# filter genes if they are expressed in at least 3 cells
sc.pp.filter_genes(adata, min_cells=3)
# Check to see how many cells/genes were filtered
adata

### <u>4C: Normalise gene counts</u>
Normalization ensures comparability between cells, and log transformation stabilizes variance.


In [ ]:
# Normalise (library-size correct) the data matrix 𝐗 to 10,000 counts per cell, so that information become comparable between cells.
sc.pp.normalize_total(adata, target_sum=1e4)

# Logarithmise the data:
sc.pp.log1p(adata)

### <u>4D: Highly variable gene selection</u>
Highly variable genes are essential for clustering and downstream analysis.


In [ ]:
# (Expects logarithimised data)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

### <u>4E: Stash the normalised data in ‘.raw’ for later use</u>

In [ ]:
# stash the normalised counts in .raw
adata.raw = adata

### <u>4F: Save before continuing</u>

In [ ]:
adata.write_h5ad("normalised.h5ad", compression="gzip")

### \*\*\*IF YOU NEED TO RESTART\*\*\*

In [ ]:
adata = sc.read_h5ad("normalised.h5ad")

## STEP 5: Dimensionality Reduction I
In single-cell data analysis, each cell is described by thousands of features (genes). This high-dimensional data is challenging to work with directly because:
1.	Noise and Redundancy: Many genes may not be relevant to the biological differences we are investigating. Others may show redundant patterns, making the data unnecessarily complex.
2.	Visualization: It's impossible to directly visualize relationships among cells in thousands of dimensions. Dimensionality reduction allows us to represent the data in fewer dimensions, making patterns easier to observe.
3.	Efficiency: Many analysis techniques, like clustering or neighbour graph construction, are computationally expensive in high dimensions. Reducing the dimensions makes these steps faster and more effective.

**Principal Component Analysis (PCA)** is a mathematical technique that addresses some of these challenges in a simple manner. It does so by:
•	Identifying the most important directions (principal components) that capture the largest amount of variance in the data.
•	Projecting the high-dimensional data into a lower-dimensional space (e.g., 10-50 dimensions) while retaining the most important information.

### <u>5A: Subset to highly variable genes</u>

In [ ]:
# Filtering so that only meaningful genes are used for PCA
adata = adata[:, adata.var["highly_variable"] == True]
adata

### <u>5B: Regress out effects of total counts per cell and %mitochondrial genes</u>
In single-cell RNA-seq data, some technical factors can introduce unwanted variation, which may overshadow the biological differences we're interested in. Two common technical confounders are:
1.	Total Counts Per Cell: The total number of RNA molecules detected in a cell can vary widely, due to factors like sequencing depth or cell size. This variability doesn't necessarily reflect true biological differences.

2.	% Mitochondrial Genes: High mitochondrial gene expression can indicate stressed or dying cells. These effects might dominate the analysis and obscure meaningful biological variation.

By regressing out these effects:
•	We remove their influence, allowing PCA (and downstream analyses) to focus on biologically relevant variation.
•	This ensures the major sources of variance in the data represent true differences among cells rather than artifacts of sequencing or cell health.


In [ ]:
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

### <u>5C: Scale each gene to unit variance. Clip values exceeding standard deviation of 10</u>
Once technical effects are removed, the next step is to scale the data so that all genes contribute equally to the analysis:
1.	Scaling to Unit Variance: Genes vary widely in their expression levels. Without scaling, highly expressed genes could dominate PCA simply due to their magnitude, even if they're not the most biologically variable. Scaling ensures that each gene is represented in the form of standard deviations of each gene’s mean, making their individual contributions to PCA more balanced.

2.	Clipping Values: Some genes may have outlier expression values that could disproportionately influence PCA. By clipping values exceeding a standard deviation of 10, we limit the impact of these extreme outliers, ensuring a more robust analysis.


In [ ]:
sc.pp.scale(adata, max_value=10)

### <u>5D: Perform Principal Component Analysis (PCA)</u>

In [ ]:
sc.tl.pca(adata, svd_solver="arpack", n_comps=50)
# visualise the variance contribution by each PC
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

## STEP 6: Dimensionality Reduction II
While dimensionality reduction techniques like PCA reduce the data to fewer dimensions, 50 in the case of this exercise, we still need a way to visualize these high-dimensional relationships in an intuitive format, such as a 2-dimensional scatter plot. 

UMAP stands for Uniform Manifold Approximation and Projection. UMAP is a dimensionality reduction technique designed for visualizing high-dimensional data in low-dimensional spaces (e.g., 2D or 3D). Unlike PCA, which focuses on capturing variance, UMAP is a nonlinear method that preserves the local structure of the data, making it particularly useful for visualizing clusters and patterns in biological datasets.

UMAP creates a visualization by:
1.	Constructing a graph of nearest neighbours in the high-dimensional space (based on the PCA-reduced data).

2.	Learning a low-dimensional representation that preserves these local relationships as much as possible.

### 6A: Compute neighbourhood graph

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **n_neighbors**: Controls how UMAP balances capturing local vs. global structure. Smaller values (e.g., 10) focus on local relationships, which is ideal for resolving tight clusters or rare cell types. Larger values (e.g., 50) give a broader view of the data, capturing more global relationships at the expense of finer details.
>
> **n_pcs**: Specifies the number of principal components (PCs) to use as input for UMAP. This parameter determines how much of the variance in the original data is captured before UMAP is applied.

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

### <u>6B: Embed the neighbourhood graph using UMAP</u>

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **min_dist**: Determines how tightly UMAP clusters points. Smaller values result in tighter clusters, while larger values spread them out.


In [ ]:
sc.tl.umap(adata, min_dist=0.3)

### <u>6C: Visualise the UMAP</u>

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **color**: column names found in the metadata (adata.obs) or specific gene names e.g. CD4


In [ ]:
sc.pl.umap(adata, color=["group"])

## STEP 7: Clustering
Clustering is a critical step that allows us to group cells with similar gene expression profiles. These clusters often correspond to distinct cell types or states, helping us uncover the biological diversity present in a tissue or condition.

We will use the leiden algorithm to cluster the cells into different groups. It is a graph-based clustering algorithm that is very popular in single-cell analysis. It is based on optimizing a modularity function that is used to detect communities in networks.

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **resolution**: parameter that can be tuned to get different levels of granularity in the clustering. Higher resolutions give finer (more) clusters while lower resolutions give broader (less) clusters. Experiment with the different resolutions!



In [ ]:
sc.tl.leiden(adata, resolution=1)

## STEP 8: Cell-type annotation

From here on, the analysis will be more focused on the biological interpretation of the clusters. We will use marker genes to annotate the clusters with cell types.

The way to approach this can vary and will first try a manual approach. Ask yourself, what are relevant genes for a celltype? What are the marker genes that you would expect to see?

# KEY TASK(S) FOR STUDENT

## Student groups need to each come up with their own sets of genes that can be used to associate each cluster in the single-cell data with a cell-type identity found in mouse spleen (at least 3 genes per identity). Several clusters can be the same cell-type.


In [ ]:
marker_genes = ["gene 1", "gene 2"]
# Plot the marker genes on a dotplot
sc.pl.dotplot(
    adata, marker_genes, groupby="leiden", standard_scale="var", color_map="Blues"
)
# Plot the marker genes on the umap
sc.pl.umap(adata, color=marker_genes, color_map="viridis")

Another way is to show using a statistical test to identify marker genes for each cluster. We will use the ‘rank_genes_groups’ function from ‘scanpy’ to identify statistically significant marker genes for each cluster.

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="leiden")

Visualise the top statistically significant marker genes for each cluster.

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **n_genes**: this adjusts how many top genes to plot


In [ ]:
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

We can also visualise this as a dot plot.

### \*\*\*OPTIONAL PARAMETER(S) TO ADJUST\*\*\*

> **n_genes**: this adjusts how many top genes to plot
>
> **color_map**: this adjust the colours. Try Reds, Greens, viridis and more! https://matplotlib.org/stable/users/explain/colors/colormaps.html
>
> **min_logfoldchange**: this controls the minimum log fold change cutoff in order for the gene to be plotted


In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=3, color_map="Blues", standard_scale="var", min_logfoldchange=1
)

## STEP 9: Mapping cell-type labels to data

Now that we have determine what cell-types each cluster correspond to, it’s time to transfer them to the data so that we can visualise them.


In [ ]:
# We will use a dictionary to map the labels
cell_types_dictionary = {
    "0": "Celltype A",
    "1": "Celltype B",
    "2": "Celltype C",
    "3": "Celltype D",
}
adata.obs["cell_type"] = [cell_types_dictionary[x] for x in adata.obs["leiden"]]

### Visualise the cell-types on the UMAP

In [ ]:
sc.pl.umap(adata, color="cell_type")
# You can also plot the legend directly on the data
sc.pl.umap(adata, color="cell_type", legend_loc="on data", legend_fontoutline=2)

# Save your progress

In [ ]:
adata.write_h5ad("processed.h5ad", compression="gzip")

If you want to load it up again, do:

In [ ]:
adata = sc.read_h5ad("processed.h5ad")
adata

# BONUS TASK FOR STUDENTS

## Student groups to compare whether cell-type proportions are different between space or earth mice.

## STEP 10: Tabulate the proportions of each cell-type

We will use the `pandas` library to do some data frame manipulation that will let us tabulate the proportions of each cell type, split by mice that were in space or on ground control.


In [ ]:
data = pd.crosstab(adata.obs["cell_type"], adata.obs["sample"])
data

We can now save this new data and extract the values to do some statistical analysis with software such as Prism, Microsoft Excel, or within Python directly.

In [ ]:
data.to_csv("proportions.csv")

You will need to refer to the metadata to match the samples to the relevant groups:


In [ ]:
adata.obs.drop_duplicates(subset=["sample"])[["sample", "group"]].reset_index(drop=True)